In [1]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import glob
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import torch.onnx
import onnx
import onnxruntime as ort
import joblib

file_paths = {
    "../input/dataset007/Seating_tilted.csv": "Seating_tilted",
    "../input/dataset007/Walking_tilted.csv": "Walking_tilted",
    "../input/dataset007/Seating_hunched.csv": "Seating_hunched",
    "../input/dataset007/Standing_hunched.csv": "Standing_hunched",
    "../input/dataset007/Walking_straight.csv": "Walking_straight",
    "../input/dataset007/Standing_tilted.csv": "Standing_tilted",
    "../input/dataset007/Walking_hunched.csv": "Walking_hunched",
    "../input/dataset007/Seating_straight.csv": "Seating_straight",
    "../input/dataset007/Standing_straight.csv": "Standing_straight"
}

dataframes = []
for file_path, label in file_paths.items():
    df = pd.read_csv(file_path)
    df['Posture_Type'] = label
    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.head()


,Flutter Timecode,Device Timecode,ax,ay,az,gx,gy,gz,Orientation,Posture,Posture_Type
0,1730965200588,282436,-0.04,0.82,0.63,22.12,0.21,-1.33,Seating,Tilted,Seating_tilted
1,1730965200588,282436,-0.04,0.82,0.63,22.12,0.21,-1.33,Seating,Tilted,Seating_tilted
2,1730965200588,282436,-0.04,0.82,0.63,22.12,0.21,-1.33,Seating,Tilted,Seating_tilted
3,1730965200588,282436,-0.04,0.82,0.63,22.12,0.21,-1.33,Seating,Tilted,Seating_tilted
4,1730965200784,282656,-0.09,0.92,0.53,27.16,7.35,-6.37,Seating,Tilted,Seating_tilted


In [3]:
len(combined_df)

370962

In [4]:
print(combined_df['Posture'].unique())
print(combined_df['Orientation'].unique())
print(combined_df['Posture_Type'].unique())

['Tilted' 'Hunched' 'Straight']
['Seating' 'Walking' 'Standing']
['Seating_tilted' 'Walking_tilted' 'Seating_hunched' 'Standing_hunched'
 'Walking_straight' 'Standing_tilted' 'Walking_hunched' 'Seating_straight'
 'Standing_straight']


In [5]:

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
'''
label_encoder_orientation = LabelEncoder()
label_encoder_posture = LabelEncoder()


combined_df['Orientation'] = label_encoder_orientation.fit_transform(combined_df['Orientation'])
combined_df['Posture'] = label_encoder_posture.fit_transform(combined_df['Posture'])
'''
label_encoder_posture_type = LabelEncoder()
combined_df['Posture_Type'] = label_encoder_posture_type.fit_transform(combined_df['Posture_Type'])
X = combined_df[['ax', 'ay', 'az', 'gx', 'gy', 'gz']]
y = combined_df['Posture_Type']

scaler = StandardScaler()
X = scaler.fit_transform(X)
joblib.dump(scaler, 'scaler.pkl')
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=seed, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=seed, shuffle=True)

X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train.values, dtype=torch.long).to(device)
X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val = torch.tensor(y_val.values, dtype=torch.long).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test.values, dtype=torch.long).to(device)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)



In [7]:

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

input_size = X_train.shape[1]
hidden_size = 512
output_size = len(label_encoder_posture_type.classes_)
model = MLP(input_size, hidden_size, output_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20


In [8]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    val_loss = 0
    model.eval()
    with torch.no_grad():
        for X_val_batch, y_val_batch in val_loader:
            val_outputs = model(X_val_batch)
            val_loss += criterion(val_outputs, y_val_batch).item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {total_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")




Epoch [1/20], Train Loss: 1.2116, Val Loss: 0.9591
Epoch [2/20], Train Loss: 1.0180, Val Loss: 0.8638
Epoch [3/20], Train Loss: 0.9563, Val Loss: 0.8115
Epoch [4/20], Train Loss: 0.9186, Val Loss: 0.7798
Epoch [5/20], Train Loss: 0.8917, Val Loss: 0.7558
Epoch [6/20], Train Loss: 0.8730, Val Loss: 0.7340
Epoch [7/20], Train Loss: 0.8554, Val Loss: 0.7134
Epoch [8/20], Train Loss: 0.8431, Val Loss: 0.7008
Epoch [9/20], Train Loss: 0.8334, Val Loss: 0.6899
Epoch [10/20], Train Loss: 0.8249, Val Loss: 0.6754
Epoch [11/20], Train Loss: 0.8181, Val Loss: 0.6695
Epoch [12/20], Train Loss: 0.8087, Val Loss: 0.6616
Epoch [13/20], Train Loss: 0.8018, Val Loss: 0.6571
Epoch [14/20], Train Loss: 0.7969, Val Loss: 0.6487
Epoch [15/20], Train Loss: 0.7904, Val Loss: 0.6441
Epoch [16/20], Train Loss: 0.7877, Val Loss: 0.6375
Epoch [17/20], Train Loss: 0.7811, Val Loss: 0.6339
Epoch [18/20], Train Loss: 0.7774, Val Loss: 0.6245
Epoch [19/20], Train Loss: 0.7744, Val Loss: 0.6253
Epoch [20/20], Train 

In [9]:

torch_model_path = "mlp_model.pth"
onnx_model_path = "mlp_model.onnx"

torch.save(model.state_dict(), torch_model_path)
print(f"Model saved in PyTorch format at {torch_model_path}")

#  ONNX format
dummy_input = torch.randn(1, X_train.shape[1]).to(device)  #  dummy input with the same shape as a batch
torch.onnx.export(
    model, 
    dummy_input, 
    onnx_model_path, 
    input_names=["input"], 
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
    opset_version=11
)
print(f"Model saved in ONNX format at {onnx_model_path}")


Model saved in PyTorch format at mlp_model.pth
Model saved in ONNX format at mlp_model.onnx


In [10]:

torch_model_path = "/kaggle/working/mlp_model.pth"
onnx_model_path = "/kaggle/working/mlp_model.onnx"

def evaluate_pytorch_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_test_batch, y_test_batch in test_loader:
            X_test_batch, y_test_batch = X_test_batch.to(device), y_test_batch.to(device)
            outputs = model(X_test_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_test_batch.size(0)
            correct += (predicted == y_test_batch).sum().item()
    print(f"Test Accuracy (PyTorch): {100 * correct / total:.2f}%")

model.load_state_dict(torch.load(torch_model_path))
model.to(device)
evaluate_pytorch_model(model, test_loader)


/tmp/ipykernel_23/3087680798.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(torch_model_path))


Test Accuracy (PyTorch): 78.24%


In [11]:

def evaluate_onnx_model(onnx_model_path, test_loader):
    ort_session = ort.InferenceSession(onnx_model_path)
    correct = 0
    total = 0
    for X_test_batch, y_test_batch in test_loader:

        X_test_batch = X_test_batch.cpu().numpy()
        y_test_batch = y_test_batch.cpu().numpy()

        ort_inputs = {"input": X_test_batch}
        ort_outs = ort_session.run(None, ort_inputs)
        predicted = torch.tensor(ort_outs[0]).argmax(dim=1)
        
        total += y_test_batch.shape[0]
        correct += (predicted.numpy() == y_test_batch).sum()
    
    print(f"Test Accuracy (ONNX): {100 * correct / total:.2f}%")

evaluate_onnx_model(onnx_model_path, test_loader)


Test Accuracy (ONNX): 78.24%
